<a href="https://colab.research.google.com/github/hellojohnkim/mmai891/blob/main/24_891_JohnKim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMAI 891: Individual Assignment

Version 1: Updated February 4, 2024

- John Kim
- 20439250
- MMAI 2024 891
- Genius Makers by Cade Metz
- Due April 21, 2024

# Preliminaries: Inspect and Set up environment

In [10]:
!pip install pattern

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 37.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 12.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.8/348.8 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332702 sha256=fd93e3f50ea1321de26beb70c19285c70511901b3a219b5677fd2732dda6badc
  Stored in dire

In [1]:
import datetime
import pandas as pd
import numpy as np

In [2]:
print(datetime.datetime.now())

2024-04-08 17:54:00.690093


In [3]:
!which python

/usr/local/bin/python


In [4]:
!python --version

Python 3.10.12


In [ ]:
!echo $PYTHONPATH

/env/python


In [ ]:
# TODO: install any packages you need to here. For example:
#pip install unidecode

In [5]:
import nltk
from nltk.corpus import stopwords
from transformers import AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
from pattern.en import suggest

ModuleNotFoundError: No module named 'pattern'

# Question 1: Sentiment Analysis via Shallow ML


**Marking**

The coding parts (i.e., 1.a, 1.b, 1.c4) will be marked based on:

- *Correctness*. Code clearly and fully performs the task specified.
- *Reproducibility*. Code is fully reproducible. I.e., you (and I) are able to run this Notebook again and again, from top to bottom, and get the same results each time.
- *Style*. Code is organized. All parts commented with clear reasoning and rationale. No old code laying around. Code easy to follow.


Parts 2 and 3 will be marked on:

- *Quality*. Response is well-justified and convincing. Responses uses facts and data where possible.
- *Style*. Response uses proper grammar, spelling, and punctuation. Response is clear and professional. Response is complete, but not overly-verbose. Response follows length guidelines.


In [6]:
# DO NOT MODIFY THIS CELL

# First, we'll read the provided labeled training data
df = pd.read_csv("https://drive.google.com/uc?export=download&id=1b8MAiN-xBdk6scM-DnufkuijDZivZJqM")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  2400 non-null   object
 1   Polarity  2400 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 37.6+ KB


In [7]:
# DO NOT MODIFY THIS CELL

# Next, we'll split it into training and test
from sklearn.model_selection import train_test_split

X = df['Sentence']
y = df['Polarity']

# So that we can evaluate how well our model is performing, we split our training data
# into training and validation.

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

## Part 1.a: Preprocessing and FE Pipeline

Clean and preprocess the data (i.e., `X_train`) as you see necessary. Extract features from the text (i.e., vectorization using BOW and/or Bag of N-Grams and/or topics and/or lexical features).


In [8]:
# Download stopwords from NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
# Initialize the tokenizer from Hugging Face's transformers
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def preprocess_text(texts):
    processed_texts = []
    for text in texts:
        # Tokenization
        tokens = tokenizer.tokenize(text)

        # Spell checking using pattern.en and case normalization
        corrected_tokens = []
        for token in tokens:
            suggestions = suggest(token)
            # Choose the most likely correction (first suggestion) and lowercase it
            corrected_token = suggestions[0][0].lower() if suggestions else token.lower()
            corrected_tokens.append(corrected_token)

        # Stop word removal
        tokens_without_sw = [token for token in corrected_tokens if token not in stop_words]

        # Reconstruct the text
        processed_text = ' '.join(tokens_without_sw)
        processed_texts.append(processed_text)

    return processed_texts

# Apply preprocessing to the training data
X_train_processed = preprocess_text(X_train)

# Initialize and fit the TF-IDF vectorizer with the preprocessed training data
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_processed)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

NameError: name 'suggest' is not defined

In [ ]:
##Perform various checks on the pre-processed data

# 1. Inspect the shape of X_train_tfidf
print("Shape of X_train_tfidf:", X_train_tfidf.shape)

# 2. View some non-zero elements from the first document's TF-IDF vector
# Convert the sparse matrix row to a dense format and print non-zero elements
first_doc_vector = X_train_tfidf[0].todense()
print("Non-zero elements in the first document's TF-IDF vector:", first_doc_vector[first_doc_vector > 0])

# 3. Check the feature names (vocabulary)
feature_names = tfidf_vectorizer.get_feature_names_out()
print("Some feature names:", feature_names[:10])  # Print the first 10 feature names

# 4. Sample document vector
# For a more readable format, you can create a DataFrame for the first document's TF-IDF vector

df_first_doc = pd.DataFrame(first_doc_vector, columns=feature_names)
# Filter out zero values for clearer visibility
df_nonzero = df_first_doc.loc[:, (df_first_doc != 0).any(axis=0)]
print("TF-IDF values for non-zero features in the first document:\n", df_nonzero.T)


Shape of X_train_tfidf: (1800, 2956)
Non-zero elements in the first document's TF-IDF vector: [[0.2861109  0.30179299 0.2861109  0.2861109  0.30179299 0.22136688
  0.23585882 0.2861109  0.26635381 0.30179299 0.26635381 0.26635381
  0.2861109 ]]
Some feature names: ['00' '10' '100' '11' '12' '13' '15' '17' '18' '1979']
TF-IDF values for non-zero features in the first document:
                     0
badly        0.286111
concentrate  0.301793
crack        0.286111
energy       0.286111
follow       0.301793
found        0.221367
hard         0.235859
honestly     0.286111
mean         0.266354
miles        0.301793
orders       0.266354
predictable  0.266354
youthful     0.286111


## Part 1.b: Model Training/Tuning/Cross Validation

Use your favorite shallow ML algorithm (such as decision trees, KNN, random forest, boosting variants) to train a classification model.  Don’t forget everything we’ve learned in the machine learning course: hyperparameter tuning, cross-validation, handling imbalanced data, etc. Make reasonable decisions and try to create the best-performing model that you can.


In [ ]:
# TODO: insert code here

## Part 1.c: Model Assessment

Use your model to predict the sentiment of the testing data. Measure the performance (e.g., accuracy, AUC, F1-score) of your model.

In [ ]:
# DO NOT MODIFY THIS CELL

test_df = pd.read_csv("https://drive.google.com/uc?export=download&id=1taoTluPBUMt9JkKAnlqDTrU49DJFpJGW")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  600 non-null    object
 1   Polarity  600 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.5+ KB


In [ ]:
# TODO: insert code here

## Part 2: Given the performance of your model, are you satisfied with the results? Explain.

Keep your response to 1000 characters or less.

TODO: Insert answer here.

## Part 3: Show five test instances in which your model was incorrect. Dive deep and find out why your model was wrong.

Keep your response to 1000 characters or less.

TODO: Insert answer here. (Feel free to create new code cells if necessary.)

# Question 2: Conceptual Understanding of the SOTA


**Marking**

The following questions will be marked on:

- *Quality*. Response is well-justified and convincing. Responses uses facts and data where possible.
- *Style*. Response uses proper grammar, spelling, and punctuation. Response is clear and professional. Response is complete, but not overly-verbose. Response follows length guidelines.


## Part 1: What is transfer learning and fine-tuning in NLP? What advantages does it have over training from scratch?


In the domain of NLP, transfer learning is a strategic approach where we utilize a model pre-trained on extensive datasets and tailor it to our specific needs. Fine-tuning further refines this process by adjusting the model to the intricacies of our targeted task. This methodology stands out for its efficiency, requiring significantly less time and data to achieve high-quality outcomes compared to traditional models built from the ground up. It leverages the foundational language understanding already embedded in the pre-trained model, providing a robust starting point and facilitating enhanced performance, even in data-scarce scenarios. This approach not only streamlines the development process but also amplifies the potential for innovative solutions in the field of NLP.

characters: 789

## Part 2: What is a Large Language Model (LLM) and what are their strengths and weaknesses?


A Large Language Model (LLM) like GPT excels in generating human-like text, understanding context, and performing diverse tasks in natural language. Strengths include their versatility across various applications, deep language understanding, and adaptability to specific domains via fine-tuning. However, LLMs face challenges like data bias, which can perpetuate ethical issues, and interpretability problems, complicating error analysis. They also demand significant computational resources, impacting accessibility and the environment. Despite their proficiency, LLMs can sometimes produce inaccurate or nonsensical outputs, especially in complex situations.

# Question 3 (Optional/Bonus): Sentiment Analysis via Deep ML

This question is optional and is worth up to 5 extra credit marks.

Use deep learning (e.g., RNNs and variants, CNNs and variants, and/or transformers) to build a model on the same dataset as Q1 and compare the results with the Shallow ML model.

You may train your own deep ML model (using, e.g., the keras library) or fine-tune a pretrained deep ML model (using, e.g., the transformers library and the Huggingface ecoystem).

In [ ]:
# TODO: Insert code here.